# Importing Packages

In [1]:
import os
from pytse_client.download import download_financial_indexes
import pandas as pd
import jalali_pandas
import math
import numpy as np

# Path

In [2]:
desktop = os.path.join(os.path.join(os.path.expanduser('~')), 'Desktop') 
industry_indices = os.path.join(desktop, "industry_indices")

# Config

In [3]:
limit = 3000

# Functions

In [4]:
def return_calculator(df):
    period_return = 1
    li = list(df)
    for day, _ in df.iteritems():
        idx = df.index.get_loc(day)
        elem = li[idx]
        if (math.isnan(elem)):
            elem = outlier_handler(elem)
            if elem == False:
                continue
        period_return = period_return * (1+elem)
    period_return = period_return - 1
    return period_return

def outlier_handler(ret):
    if (math.isnan(ret)):
        return False


In [5]:
download_financial_indexes(symbols="all", write_to_csv=True, base_path=industry_indices)

{'پيمانكاري':       value       date
 0     327.9 2008-12-05
 1     327.2 2008-12-06
 2     327.5 2008-12-07
 3     327.7 2008-12-08
 4     328.1 2008-12-10
 ...     ...        ...
 3389  919.7 2022-12-25
 3390  919.7 2022-12-26
 3391  919.7 2022-12-28
 3392  919.7 2022-12-31
 3393  919.7 2023-01-01
 
 [3394 rows x 2 columns],
 'انتشار و چاپ':          value       date
 0       7854.6 2008-12-05
 1       7854.6 2008-12-06
 2       7854.6 2008-12-07
 3       7854.6 2008-12-08
 4       7854.6 2008-12-10
 ...        ...        ...
 3393  909040.0 2022-12-24
 3394  943657.0 2022-12-25
 3395  949626.0 2022-12-26
 3396  986495.0 2022-12-28
 3397  939401.0 2022-12-31
 
 [3398 rows x 2 columns],
 'محصولات فلزي':          value       date
 0       1244.2 2008-12-05
 1       1244.2 2008-12-06
 2       1244.2 2008-12-07
 3       1244.2 2008-12-08
 4       1244.2 2008-12-10
 ...        ...        ...
 3393  154117.0 2022-12-24
 3394  157321.0 2022-12-25
 3395  157792.0 2022-12-26
 3396  162555.0 2

In [6]:
list_of_files = os.listdir(industry_indices)
to_delete_indices = ["شاخص 30 شركت بزرگ.csv","شاخص آزاد شناور.csv","شاخص بازار اول.csv","شاخص بازار دوم.csv",
             "شاخص بازده نقدي قيمت.csv", "شاخص صنعت.csv", "شاخص قيمت (هم وزن).csv", "شاخص قيمت 50 شركت.csv",
             "شاخص قيمت(وزنيارزشي).csv", "شاخص50شركت فعالتر.csv", "شاخص كل.csv", "شاخص كل (هم وزن).csv"
             ]
defunct_sectors = ["مبلمان.csv", "ابزار پزشكي.csv" , "پيمانكاري.csv"]

In [7]:
sectors_li = [item for item in list_of_files if item not in to_delete_indices]
sectors_li = [item for item in sectors_li if item not in defunct_sectors]

In [8]:
for f in sectors_li:
    data = pd.read_csv(industry_indices+ "\\" + f)
    if len(data) < limit:
        sectors_li.remove(f)
    print(len(data),f)

1452 اداره بازارهاي مالي.csv
1275 اطلاعات و ارتباطات.csv
3398 انتشار و چاپ.csv
3398 بانكها.csv
3193 بيمه و بازنشسته.csv
1853 تامين آب، برق، گاز.csv
930 خرده فروشي به جز وسايل نقليه.csv
3398 دستگاههاي برقي.csv
3398 ذغال سنگ.csv
3398 راديويي.csv
3398 رايانه.csv
3399 زراعت.csv
3398 ساير معادن.csv
3398 سايرمالي.csv
3398 سرمايه گذاريها.csv
3398 سيمان.csv
3399 شيميايي.csv
3398 غذايي بجز قند.csv
3398 فراورده نفتي.csv
3398 فلزات اساسي.csv
3398 فني مهندسي.csv
3398 قند و شكر.csv
3398 كاشي و سراميك.csv
3398 كانه فلزي.csv
3398 كاني غيرفلزي.csv
3398 لاستيك.csv
3398 ماشين آلات.csv
3398 مالي.csv
3398 محصولات فلزي.csv
3398 محصولات كاغذ.csv
3397 محصولات چرمي.csv
3398 محصولات چوبي.csv
3398 منسوجات.csv
3399 مواد دارويي.csv
3398 وسايل ارتباطي.csv
3398 چند رشته اي ص.csv


In [9]:
len(sectors_li)

36

In [10]:
sectors_li[0]

'استخراج نفت جزكشف.csv'

In [11]:
data = pd.read_csv(industry_indices+ "\\" + sectors_li[0])
data['date'] = pd.to_datetime( data['date'],    format='%Y-%m-%d')
data["jdate"] = data["date"].jalali.to_jalali()
data["month"] = data["jdate"].jalali.month
data["year"] = data["jdate"].jalali.year
data["weekday"] = data["jdate"].jalali.weekday
#data.set_index('date' ,inplace=True)
data['value'] = data['value'].pct_change()
data['value'].values[data['value'].values > 0.05] = 0.05
data['value'].values[data['value'].values < -0.05] = -0.05
#weekly_data = data.resample("W").sum()

first_years_month = data["month"][0]
first_year = data["year"][0]
last_years_month = 12
last_year = 1396

empty_li_month = []
empty_li_year = []
for year in range(first_year, last_year + 1):

        if year == first_year:
            empty_li_month += [i for i in range(first_years_month,13)]
            empty_li_year += [year] * (13-first_years_month)

        elif year == last_year:
            empty_li_month += [i for i in range(1,last_years_month+1)]
            empty_li_year += [year] * (last_years_month)

        else:
            empty_li_month += [i for i in range(1,13)]
            empty_li_year += [year] * 12

monthly_data = pd.DataFrame(columns=["return","year","month","return_1m","return_3m","return_6m"])
monthly_data["year"] = empty_li_year
monthly_data["month"] = empty_li_month

for year in range(first_year, last_year + 1):

    if year == first_year:
        for month in range(first_years_month, 13):
            idx = monthly_data[(monthly_data.year == year) & (monthly_data.month == month)].index.item()
            ret = return_calculator(data[(data["month"] == month) & (data["year"] == year)]["value"])
            monthly_data["return"][idx] = ret

    elif year == last_year:
        for month in range(1, last_years_month + 1):
            idx = monthly_data[(monthly_data.year == year) & (monthly_data.month == month)].index.item()
            ret = return_calculator(data[(data["month"] == month) & (data["year"] == year)]["value"])
            monthly_data["return"][idx] = ret

    else:
        for month in range(1, 13):
            idx = monthly_data[(monthly_data.year == year) & (monthly_data.month == month)].index.item()
            ret = return_calculator(data[(data["month"] == month) & (data["year"] == year)]["value"])
            monthly_data["return"][idx] = ret

monthly_data["return"] = monthly_data["return"] * 100
monthly_data["return_1m"] = monthly_data["return"].shift(1)
monthly_data["return_3m"] = monthly_data["return"].shift(3)
monthly_data["return_6m"] = monthly_data["return"].shift(6)
monthly_data = monthly_data.dropna().reset_index()
monthly_data = monthly_data.drop(['index'], axis=1)

C:\Users\Amin\AppData\Local\Temp\ipykernel_1588\611742881.py:4: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for day, _ in df.iteritems():
C:\Users\Amin\AppData\Local\Temp\ipykernel_1588\152148241.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_data["return"][idx] = ret
C:\Users\Amin\AppData\Local\Temp\ipykernel_1588\152148241.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_data["return"][idx] = ret
C:\Users\Amin\AppData\Local\Temp\ipykernel_1588\152148241.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [12]:
sectors_li[0]

'استخراج نفت جزكشف.csv'

In [13]:
index_data = pd.read_csv(industry_indices+ "\\" + "شاخص كل.csv")

index_data['date'] = pd.to_datetime( index_data['date'],    format='%Y-%m-%d')
index_data["jdate"] = index_data["date"].jalali.to_jalali()
index_data["month"] = index_data["jdate"].jalali.month
index_data["year"] = index_data["jdate"].jalali.year
index_data["weekday"] = index_data["jdate"].jalali.weekday
#data.set_index('date' ,inplace=True)
index_data['value'] = index_data['value'].pct_change()
index_data['value'].values[index_data['value'].values > 0.05] = 0.05
index_data['value'].values[index_data['value'].values < -0.05] = -0.05
#weekly_data = data.resample("W").sum()

first_years_month = index_data["month"][0]
first_year = index_data["year"][0]
last_years_month = 12
last_year = 1396

empty_li_month = []
empty_li_year = []
for year in range(first_year, last_year + 1):

        if year == first_year:
            empty_li_month += [i for i in range(first_years_month,13)]
            empty_li_year += [year] * (13-first_years_month)

        elif year == last_year:
            empty_li_month += [i for i in range(1,last_years_month+1)]
            empty_li_year += [year] * (last_years_month)

        else:
            empty_li_month += [i for i in range(1,13)]
            empty_li_year += [year] * 12

index_monthly_data = pd.DataFrame(columns=["index_return","year","month","index_return_1m","index_return_3m","index_return_6m"])
index_monthly_data["year"] = empty_li_year
index_monthly_data["month"] = empty_li_month

for year in range(first_year, last_year + 1):

    if year == first_year:
        for month in range(first_years_month, 13):
            idx = index_monthly_data[(index_monthly_data.year == year) & (index_monthly_data.month == month)].index.item()
            ret = return_calculator(index_data[(index_data["month"] == month) & (index_data["year"] == year)]["value"])
            index_monthly_data["index_return"][idx] = ret

    elif year == last_year:
        for month in range(1, last_years_month + 1):
            idx = index_monthly_data[(index_monthly_data.year == year) & (index_monthly_data.month == month)].index.item()
            ret = return_calculator(index_data[(index_data["month"] == month) & (index_data["year"] == year)]["value"])
            index_monthly_data["index_return"][idx] = ret

    else:
        for month in range(1, 13):
            idx = index_monthly_data[(index_monthly_data.year == year) & (index_monthly_data.month == month)].index.item()
            ret = return_calculator(index_data[(index_data["month"] == month) & (index_data["year"] == year)]["value"])
            index_monthly_data["index_return"][idx] = ret

index_monthly_data["index_return"] = index_monthly_data["index_return"] * 100
index_monthly_data["index_return_1m"] = index_monthly_data["index_return"].shift(1)
index_monthly_data["index_return_3m"] = index_monthly_data["index_return"].shift(3)
index_monthly_data["index_return_6m"] = index_monthly_data["index_return"].shift(6)
index_monthly_data = index_monthly_data.dropna().reset_index()
index_monthly_data = index_monthly_data.drop(['index'], axis=1)

C:\Users\Amin\AppData\Local\Temp\ipykernel_1588\611742881.py:4: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for day, _ in df.iteritems():
C:\Users\Amin\AppData\Local\Temp\ipykernel_1588\3988837126.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  index_monthly_data["index_return"][idx] = ret
C:\Users\Amin\AppData\Local\Temp\ipykernel_1588\3988837126.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  index_monthly_data["index_return"][idx] = ret
C:\Users\Amin\AppData\Local\Temp\ipykernel_1588\3988837126.py:51: SettingWithCopyWarning: 
A value is trying to be

In [14]:
index_data.to_csv("test.csv")

In [15]:
monthly_data

,return,year,month,return_1m,return_3m,return_6m
0,-4.573687,1388,11,6.751055,0.0,41.165049
1,10.486381,1388,12,-4.573687,-18.828052,7.526218
2,6.81458,1389,1,10.486381,6.751055,23.012048
3,9.742829,1389,2,6.81458,-4.573687,0.0
4,-17.440288,1389,3,9.742829,10.486381,-18.828052
...,...,...,...,...,...,...
93,-0.271186,1396,8,-4.386344,0.0,-7.694407
94,5.393157,1396,9,-0.271186,-14.860573,-10.094591
95,-15.824554,1396,10,5.393157,-4.386344,-0.739767
96,-8.284486,1396,11,-15.824554,-0.271186,0.0


In [16]:
index_monthly_data

,index_return,year,month,index_return_1m,index_return_3m,index_return_6m
0,2.861968,1388,3,6.755859,-4.895781,-4.236179
1,2.849304,1388,4,2.861968,5.833176,-4.39986
2,10.850434,1388,5,2.849304,6.755859,-1.072348
3,8.343123,1388,6,10.850434,2.861968,-4.895781
4,6.067179,1388,7,8.343123,2.849304,5.833176
...,...,...,...,...,...,...
101,2.653093,1396,8,0.755431,2.109313,3.232746
102,7.585503,1396,9,2.653093,4.199708,-3.027071
103,3.464191,1396,10,7.585503,0.755431,2.457065
104,-0.676704,1396,11,3.464191,2.653093,2.109313


In [17]:
merged_data = monthly_data.merge(index_monthly_data, on=["month","year"])

In [116]:
merged_data["month_return"] = merged_data["return"] - merged_data["index_return"]
merged_data["1month_return"] = merged_data["return_1m"] - merged_data["index_return_1m"]
merged_data["3month_return"] = merged_data["return_3m"] - merged_data["index_return_3m"]
merged_data["6month_return"] = merged_data["return_6m"] - merged_data["index_return_6m"]
merged_data["1m_3m"] = np.array((merged_data["1month_return"] + merged_data["3month_return"])/150,dtype=float)
merged_data["1m_3m"] = 1000 * ((1/(1 + np.exp(-merged_data["1m_3m"]))) - 0.5) 
merged_data["1m_6m"] = np.array((merged_data["1month_return"] + merged_data["6month_return"])/150,dtype=float)
merged_data["1m_6m"] = 1000 * ((1/(1 + np.exp(-merged_data["1m_6m"]))) - 0.5) 